In [ ]:
from custom_model import EmotionClassifier
from custom_model import EmotionBERT
import torch

def main():
    
    classifier = EmotionClassifier(device='cpu')
    
    
    try:
        
        with open("dict.txt", "r") as f:
            emotion_labels = [line.strip() for line in f]
        
        
        classifier._setup_labels(emotion_labels)
        
        
        classifier.model = EmotionBERT(n_classes=len(emotion_labels))
        
        
        classifier.model.load_state_dict(torch.load('best_model_state.pth'))
        
        
        example_text = "i want to cry, depression is coming, life is worth nothing"
        predicted_labels = classifier.predict(example_text)
        
        print(f"Tekst: '{example_text}'")
        print(f"Przewidywane emocje: {predicted_labels}")
        
    except FileNotFoundError:
        print("Błąd: Nie znaleziono pliku modelu 'best_model_state.pth' lub 'dict.txt'")
    except Exception as e:
        print(f"Wystąpił błąd: {str(e)}")

if __name__ == "__main__":
    main()

Labels setup: 28 unique labels found: ['admiration', 'amusement', 'anger', 'annoyance', 'approval', 'caring', 'confusion', 'curiosity', 'desire', 'disappointment', 'disapproval', 'disgust', 'embarrassment', 'excitement', 'fear', 'gratitude', 'grief', 'joy', 'love', 'nervousness', 'neutral', 'optimism', 'pride', 'realization', 'relief', 'remorse', 'sadness', 'surprise']
Tekst: 'i want to cry, depression is coming, life is worth nothing'
Przewidywane emocje: {'disappointment': 0.7149401307106018}
